In [1]:
import re
import os
import string

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import pandas as pd
import numpy as np

dictionary = {}

docIdMap = {}
kata = []
base = []
child = []
namaDokumen = ""

In [2]:
def proses_pertama(path):
    docId = 1
    """
            Membuat pengulangan sesuai jumlah data txt pada suatu folder
    """
    fileList = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    print(fileList)
    fileobj = open('frekuensi.txt', 'w')
    for eachFile in fileList:
        position = 1
        count = 0
        """
        Di variable bawah ini nanti akan dimasukkan id File dan nama File txtnya
        """
        docIdMap[docId] = eachFile
        """
        DI variable bawah ini kita akan mendapatkan kalimat dari tiap barisnya
        """
        lines = [line.rstrip('\n') for line in open(path + "/" + eachFile)]
#         print(lines)        
#         print("================buat pengulangan")
        for eachLine in lines:
            """
            case folding
            """
            eachLine = eachLine.lower()
            eachLine = re.sub(r"\d+", "", eachLine)
            eachLine = eachLine.translate(str.maketrans("","",string.punctuation))
            #hapus simbol
            eachLine = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',eachLine)
            eachLine = re.sub('rt @[^\s]+',' ', eachLine)
            eachLine = re.sub('@[^\s]+',' ',eachLine)
            eachLine = re.sub(r'[^\x00-\x7F]+',' ', eachLine)
            #Remove additional white spaces
            eachLine = re.sub('[\s]+', ' ', eachLine)
            eachLine = re.sub(r'#([^\s]+)', r'\1', eachLine)
            eachLine = eachLine.strip()
            """
            stop words
            """
            stop_words = set(stopwords.words('english')) 
  
            word_tokens = word_tokenize(eachLine) 
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
#             print("==========================================STEMMING")
            """
            Stemming
            """
            ps = PorterStemmer()
            for k in filtered_sentence:
                kata.append(ps.stem(k))
#             for word in filtered_sentence:
                word = ps.stem(k)
                """
                    memasukkan ke dict,
                    nantinya biar dihitung frekeunsi tiap corpus
                """
                if (word.lower() in dictionary):
                    postingList = dictionary[word.lower()]
                    if (docId in postingList):
                        postingList[docId].append(position)
                        position = position + 1
                    else:
                        postingList[docId] = [position]
                        position = position + 1
                else:
                    dictionary[word.lower()] = {docId: [position]}
                    position = position + 1
#             print("==========================================END")
                    
        docId = docId + 1
            
    length_dict = {key: len(value) for key, value in dictionary.items()}
    """
        PROSES MENGHITUNG FREKUENSI TIAP FILE CORPUS
        dimasukkan ke dalam array child
    """
    for key in dictionary:
#         print(key)
        child.append(key)
#         for s, value in dictionary[key].items():
        for sw in docIdMap:
            pass
            for s, value in dictionary[key].items():
                if (str(s) in str(sw)):
#                     print("pada dok "+str(sw) +" "+str(len(dictionary[key][s])))
                    child.append(len(dictionary[key][s]))
                    break
            else:
#                 print("pada dok gagal "+str(sw) +" 0")
                child.append(0)
        base.append(child)

    """
    Buat file semua term pada semua corpus !
    """
    for w in length_dict:
        # ini nanti dibuat menangkap jumlah kata di suatu file txt
        fileobj.write(w +"   |   "+str(length_dict[w]))
        fileobj.write("\n")
    fileobj.close()
    print("================================================")
#     print(child)

In [3]:
def main():
    AlamatDir = "test"
    queryFile = "query"
    proses_pertama(AlamatDir)

In [4]:
if __name__ == '__main__':
    main()

['data1.txt', 'data3.txt', 'data2.txt']


In [10]:
data = [child[i:i+4] for i in range(0, len(child),4)]
kategori = ['business','entertainment','politics']
kolom = []
p = []
kolom.append('Term / kata')
for x in docIdMap:
    p.append(str(kategori[x-1]))
    kolom.append(str(kategori[x-1]))
print(docIdMap)
# Create the pandas DataFrame 
df = pd.DataFrame(data, columns = kolom) 
export = df.sort_values(p, ascending=False)
export.head()
# print dataframe. 
# print(pd.concat([pd.DataFrame({})] + [df], axis=1))

{1: 'data1.txt', 2: 'data3.txt', 3: 'data2.txt'}


,Term / kata,business,entertainment,politics
134,said,151,257,100
211,us,96,8,37
10,bn,93,12,2
132,year,89,46,73
181,market,59,13,2


Jika kita ingin mencari term/kata

In [6]:
df[df['Term / kata'].str.contains("book")]

,Term / kata,business,entertainment,politics
3,book,2,15,12
4918,songbook,0,0,1
5029,bookmak,0,0,2
5270,booker,0,0,1


Jika ingin export ke csv file

In [7]:
export.to_csv (r'hasil-baru.csv', index = False, header=True)

In [9]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = kolom

for z in data:
    x.add_row(z)

# print(x)
# print(x.get_string(sortby="Term / kata", reversesort = False))